In [94]:
# imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import psycopg2
from sqlalchemy import create_engine, text

%matplotlib inline
%load_ext sql
%config SqlMagic.displaylimit = 30
%config SqlMagic.autolimit = 30

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [93]:
# Connecting to the database

import json
import urllib.parse
with open('credentials/credentials.json') as f:
    login = json.load(f)
    username = login['user']
    password = urllib.parse.quote(login['password'])
    host = login['host']
    port = login['port']

In [95]:
conn = create_engine(f'postgresql://{username}:{password}@{host}:{port}/football_transfermarkt')

In [91]:
data_appearances = pd.read_csv("data/appearances.csv")
data_club_games = pd.read_csv("data/club_games.csv")
data_clubs = pd.read_csv("data/clubs.csv")
data_competitions = pd.read_csv("data/competitions.csv")
data_games = pd.read_csv("data/games.csv")
data_player_valuations = pd.read_csv("data/player_valuations.csv")
data_players = pd.read_csv("data/players.csv")

In [21]:
data_appearances.head()

,appearance_id,game_id,player_id,player_club_id,date,player_pretty_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2483937_52453,2483937,52453,28095,2014-08-08,Haris Handzic,RU1,0,0,0,0,90
1,2479929_67064,2479929,67064,28095,2014-08-03,Felicio Brown Forbes,RU1,0,0,0,0,90
2,2483937_67064,2483937,67064,28095,2014-08-08,Felicio Brown Forbes,RU1,0,0,0,0,90
3,2484582_67064,2484582,67064,28095,2014-08-13,Felicio Brown Forbes,RU1,0,0,0,0,55
4,2485965_67064,2485965,67064,28095,2014-08-16,Felicio Brown Forbes,RU1,0,0,0,0,90


In [22]:
data_club_games.head()

,club_id,game_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,58,2219794,3,-1,John van den Brom,498,2,-1,Peter Maes,Home,1
1,131,2244388,3,-1,Tito Vilanova,418,2,-1,José Mourinho,Home,1
2,383,2211607,4,-1,Dick Advocaat,610,2,-1,Frank de Boer,Home,1
3,720,2252846,1,-1,Vítor Pereira,2990,0,-1,Pedro Emanuel,Home,1
4,27,2229332,2,-1,Jupp Heynckes,16,1,-1,Jürgen Klopp,Home,1


In [23]:
data_clubs.head()

,club_id,name,pretty_name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,url
0,1032,fc-reading,Fc Reading,GB1,33.66,26,25.9,12,46.2,6,Select Car Leasing Stadium,24161,+£8.37m,Brian McDermott,https://www.transfermarkt.co.uk/fc-reading/sta...
1,2323,orduspor,Orduspor,TR1,NaN,0,NaN,0,NaN,0,19 Eylül Stadyum,11024,+-0,Héctor Cúper,https://www.transfermarkt.co.uk/orduspor/start...
2,1387,acn-siena-1904,Acn Siena 1904,IT1,4.32,30,26.2,6,20.0,2,Artemio Franchi,15373,£-6Th.,Serse Cosmi,https://www.transfermarkt.co.uk/acn-siena-1904...
3,1071,wigan-athletic,Wigan Athletic,GB1,12.38,29,26.5,14,48.3,6,DW Stadium,25133,£-140Th.,Roberto Martínez,https://www.transfermarkt.co.uk/wigan-athletic...
4,2703,spartak-vladikavkaz,Spartak Vladikavkaz,RU1,NaN,1,20.0,0,NaN,0,Republican Stadium Spartak,32464,+-0,Vladimir Gazzaev,https://www.transfermarkt.co.uk/spartak-vladik...


In [24]:
data_competitions.head()

,competition_id,pretty_name,type,sub_type,country_id,country_name,country_latitude,country_longitude,domestic_league_code,name,confederation,url
0,L1,Bundesliga,domestic_league,first_tier,40,Germany,51.163818,10.447831,L1,bundesliga,europa,https://www.transfermarkt.co.uk/bundesliga/sta...
1,DFB,Dfb Pokal,domestic_cup,domestic_cup,40,Germany,51.163818,10.447831,L1,dfb-pokal,europa,https://www.transfermarkt.co.uk/dfb-pokal/star...
2,DFL,Dfl Supercup,other,domestic_super_cup,40,Germany,51.163818,10.447831,L1,dfl-supercup,europa,https://www.transfermarkt.co.uk/dfl-supercup/s...
3,NL1,Eredivisie,domestic_league,first_tier,122,Netherlands,52.243498,5.634323,NL1,eredivisie,europa,https://www.transfermarkt.co.uk/eredivisie/sta...
4,NLP,Toto Knvb Beker,domestic_cup,domestic_cup,122,Netherlands,52.243498,5.634323,NL1,toto-knvb-beker,europa,https://www.transfermarkt.co.uk/toto-knvb-beke...


In [25]:
data_games.head()

,game_id,competition_id,competition_type,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,...,home_club_position,away_club_position,club_home_pretty_name,club_away_pretty_name,home_club_manager_name,away_club_manager_name,stadium,attendance,referee,url
0,2219794,BESC,other,2011,Final,2012-07-22,58,498,3,2,...,-1,-1,Rsc Anderlecht,Ksc Lokeren,John van den Brom,Peter Maes,Constant Vanden Stock Stadion,14485,Laurent Colemonts,https://www.transfermarkt.co.uk/spielbericht/i...
1,2244388,SUC,other,2012,final 1st leg,2012-08-22,131,418,3,2,...,-1,-1,Fc Barcelona,Real Madrid,Tito Vilanova,José Mourinho,Spotify Camp Nou,91728,Carlos Clos Gómez,https://www.transfermarkt.co.uk/spielbericht/i...
2,2211607,NLSC,other,2012,Final,2012-08-05,383,610,4,2,...,-1,-1,Psv Eindhoven,Ajax Amsterdam,Dick Advocaat,Frank de Boer,Johan Cruijff ArenA,50000,Björn Kuipers,https://www.transfermarkt.co.uk/spielbericht/i...
3,2252846,POSU,other,2012,Final,2012-08-11,720,2990,1,0,...,-1,-1,Fc Porto,Academica Coimbra,Vítor Pereira,Pedro Emanuel,Estádio do Dragão,26000,Olegário Benquerença,https://www.transfermarkt.co.uk/spielbericht/i...
4,2229332,DFL,other,2012,Final,2012-08-12,27,16,2,1,...,-1,-1,Fc Bayern Munchen,Borussia Dortmund,Jupp Heynckes,Jürgen Klopp,Allianz Arena,69000,Michael Weiner,https://www.transfermarkt.co.uk/spielbericht/i...


In [26]:
data_player_valuations.head()

,date,datetime,dateweek,player_id,current_club_id,market_value,player_club_domestic_competition_id
0,2013-07-22,2013-07-22,2013-07-22,254016,855,90000,FR1
1,2014-01-14,2014-01-14,2014-01-13,254016,855,90000,FR1
2,2008-08-25,2008-08-25,2008-08-25,51053,23,90000,L1
3,2011-05-30,2011-05-30,2011-05-30,51053,23,90000,L1
4,2011-08-26,2011-08-26,2011-08-22,51053,23,225000,L1


In [27]:
data_players.head()

,player_id,pretty_name,club_id,club_pretty_name,current_club_id,country_of_citizenship,country_of_birth,city_of_birth,date_of_birth,position,...,height_in_cm,market_value_in_gbp,highest_market_value_in_gbp,agent_name,contract_expiration_date,domestic_competition_id,club_name,image_url,last_season,url
0,254016,Arthur Delalande,855,Ea Guingamp,855,France,France,Saint-Malo,1992-05-18,Midfield,...,186,NaN,90000.0,NaN,NaN,FR1,ea-guingamp,NaN,2013,https://www.transfermarkt.co.uk/arthur-delalan...
1,51053,Daniel Davari,23,Eintracht Braunschweig,23,Iran,Germany,Gießen,1988-01-06,Goalkeeper,...,192,135000.0,1130000.0,NG360,2022-06-30,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/daniel-davari/...
2,31451,Torsten Oehrl,23,Eintracht Braunschweig,23,Germany,Germany,Lichtenfels,1986-01-07,Attack,...,192,NaN,1130000.0,NaN,NaN,L1,eintracht-braunschweig,NaN,2013,https://www.transfermarkt.co.uk/torsten-oehrl/...
3,44622,Vladimir Kisenkov,3691,Tom Tomsk,3691,Russia,UdSSR,Kaluga,1981-10-08,Defender,...,182,NaN,720000.0,NaN,NaN,RU1,tom-tomsk,NaN,2013,https://www.transfermarkt.co.uk/vladimir-kisen...
4,30802,Oscar Diaz,3302,Ud Almeria,3302,Spain,Spain,Madrid,1984-04-24,Attack,...,183,NaN,1080000.0,Pedro Bravo - Consulting,2022-06-30,ES1,ud-almeria,NaN,2013,https://www.transfermarkt.co.uk/oscar-diaz/pro...


In [56]:
# players of interest
interest = ['(?i).*\sMessi$', '(?i).*\sronaldo$', '(?i).*\smbappe$', '(?i).*\sneymar$']

In [76]:
temp = data_appearances.groupby(["player_id"])["goals"].sum().sort_values(ascending = False)